In [23]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [24]:
import json
import os
import mp
from tqdm import tqdm

timestamps = [i * 0.02 for i in range(1500 + 1)]

In [2]:
with open('prepare-force-alignment.json') as fopen:
    d = json.load(fopen)
len(d)

510940

In [18]:
def loop(rows):
    rows, _ = rows
    result = []
    for r in tqdm(rows):
        f = r['audio_filename']
        new_f = f.replace('/', '_').replace('.mp3', '.json').replace('.wav', '.json')
        filename = os.path.join('force_alignment', new_f)
        if not os.path.exists(filename):
            continue
        
        with open(filename) as fopen:
            d_ = json.load(fopen)
            
        c = d_['word_timestamps']
        ts = []
        for c_ in c:
            start = min(timestamps, key=lambda t: abs(t - (c_['start'])))
            end = min(timestamps, key=lambda t: abs(t - (c_['end'])))
            w = c_['text']
            t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
            ts.append(t)

        ts = ''.join(ts)
        new_text = text = f"<|startoftranscript|><|ms|><|transcribeprecise|>{ts}<|endoftext|>"
        
        result.append({
            'new_text': new_text,
            'audio_filename': f.split('speech-instructions/')[1]
        })
    return result

In [19]:
r = loop((d[:100], 0))

100%|████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 646.45it/s]


In [25]:
r[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.06|> Bagaimanakah<|0.76|><|0.82|> kita<|0.98|><|1.04|> boleh<|1.30|><|1.38|> mengendalikan<|2.08|><|2.18|> ralat<|2.46|><|2.84|> pada<|3.00|><|3.08|> laman<|3.32|><|3.44|> web<|3.56|><|3.62|> menggunakan<|4.06|><|4.14|> JavaScript?<|4.88|><|endoftext|>',
 'audio_filename': 'short-coding-2/0.mp3'}

In [26]:
r = mp.multiprocessing(d, loop, cores = 10)

100%|████████████████████████████████████████████████████████████████████████████████| 51094/51094 [02:38<00:00, 321.85it/s]


In [27]:
len(r)

510940

In [28]:
r[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.06|> Bagaimanakah<|0.76|><|0.82|> kita<|0.98|><|1.04|> boleh<|1.30|><|1.38|> mengendalikan<|2.08|><|2.18|> ralat<|2.46|><|2.84|> pada<|3.00|><|3.08|> laman<|3.32|><|3.44|> web<|3.56|><|3.62|> menggunakan<|4.06|><|4.14|> JavaScript?<|4.88|><|endoftext|>',
 'audio_filename': 'short-coding-2/0.mp3'}

In [29]:
import pandas as pd

pd.DataFrame(r).to_parquet('gather-force-alignment-word-timestamps.parquet')

In [31]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="gather-force-alignment-word-timestamps.parquet",
    path_in_repo="data/malaysian_speech_instructions_word_timestamp-00000-of-00001.parquet",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

gather-force-alignment-word-timestamps.parquet:   0%|          | 0.00/86.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/3caa9d2b120135f6cfb18a1b0a1de47bdf4c264b', commit_message='Upload data/malaysian_speech_instructions_word_timestamp-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='3caa9d2b120135f6cfb18a1b0a1de47bdf4c264b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)